In [16]:
import pandas as pd
import re
from pathlib import Path
from dotenv import load_dotenv
import os

In [17]:
load_dotenv()

TSV = Path(os.getenv("VIDEO_DATA_TSV")).resolve()
TARGET_DIR = Path(os.getenv("CHAPTERS_DIR")).resolve()
assert TSV.is_file(), "Could not find TSV."
assert TARGET_DIR.is_dir(), "Could not find chapters directory."

In [18]:
def extract_sections(text: str) -> dict[str, str]:
    """Extracts numbered text sections and optionally Vorfilm section"""

    chapters = []
    contents = []

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Optional Vorfilm
    vorfilm_match = re.match(r"(Vorfilm:)(.*?)(?=\s\d{1,2}\.\s)", text)
    if vorfilm_match:
        chapters.append("Vorfilm")
        contents.append(vorfilm_match.group(2).strip())
        text = text[vorfilm_match.end():].strip()

    # Candidate chapter markers (still permissive)
    pattern = re.compile(r"\b(\d{1,2})\.\s")
    matches = list(pattern.finditer(text))

    # Filter by strict ascending sequence
    valid_matches = []
    expected = 1

    for m in matches:
        num = int(m.group(1))
        if num == expected:
            valid_matches.append(m)
            expected += 1

    # Split text using validated matches
    for i, match in enumerate(valid_matches):
        start = match.start()
        end = (
            valid_matches[i + 1].start()
            if i + 1 < len(valid_matches)
            else len(text)
        )

        chapter_num = match.group(1)
        chapter_text = text[start:end].strip()

        chapters.append(chapter_num)
        contents.append(chapter_text)

    return {
        "chapter": chapters,
        "content": contents
    }



def save_chapters(row: pd.Series) -> None:
    print(row)

    assert isinstance(
        row["content"], str
    ), "Function must be applied to rows which have non-null content"
    filename = str(row["episode"]) + "_" + str(row["year"]) + "_chapters.tsv"
    target_filepath = Path(TARGET_DIR / filename).resolve()

    result = extract_sections(row["content"])
    result_df = pd.DataFrame(result)
    print(result_df)
    result_df.to_csv(target_filepath, index=False, sep="\t")

In [19]:
df = pd.read_csv(TSV, sep="\t")
df.columns

Index(['episode', 'year', 'file', 'length', 'fps', 'frames', 'description',
       'url_bundesarchiv', 'url_m3u8', 'keywords', 'content'],
      dtype='object')

In [20]:
df[df["content"].isna()].count()

episode             44
year                44
file                44
length              44
fps                 44
frames              44
description         44
url_bundesarchiv    44
url_m3u8            44
keywords            44
content              0
dtype: int64

In [21]:
df[(df["content"].notna()) & df["content"].str.startswith("1.")].count()

episode             148
year                148
file                145
length              145
fps                 145
frames              145
description         148
url_bundesarchiv    148
url_m3u8            145
keywords            148
content             148
dtype: int64

In [22]:
df["content"].iloc[4]

'Vorfilm: Verwundetenversorgung der deutschen Wehrmacht Straßenkampf in einer französischen Stadt: Verwundete werden in Sicherheit gebracht und mit einem Transportflugzeug (Junkers Ju 52) aus dem Kampfgebiet geflogen. Schwestern und Sanitäter des Deutschen Roten Kreuzes (DRK) im Einsatz, bei der Pflege und der Betreuung der Verwundeten und dem Transport mit Zügen und Sankas. Betreuung in einem Lazarett in Deutschland, Plakat des Kriegshilfswerks des DRK zum Spendenaufruf. Vorfilm: Tran und Helle, Benzinverbrauch Tran und Helle unterhalten sich in einer Gaststätte mit einem Mann und einer Frau. Es geht um den Benzinverbrauch beim Autofahren. Es wird darauf hingewiesen, dass man keinerlei unnötige Fahrten machen solle, da das Benzin für den Krieg benötigt wird. (00:03:35) 1. Staatsbesuche bei Hitler in Berlin und München, Deutsches Reich, 1940 Der italienische Außenminister Graf Ciano Galleazzo trifft mit dem Zug in Berlin ein und wird von Reichaußenminister Joachim von Ribbentrop begrüß

In [23]:
df[df["content"].notna()].apply(lambda row: save_chapters(row), axis=1)

episode                                                           511
year                                                             1940
file                /run/media/elisa/6DFB-5C3E/downloads/511_1940.mp4
length                                                         2564.0
fps                                                              25.0
frames                                                        64123.0
description         1. Luftkrieg über Le Havre, Frankreich, 1940  ...
url_bundesarchiv    https://digitaler-lesesaal.bundesarchiv.de/vid...
url_m3u8            https://digitaler-lesesaal.bundesarchiv.de/lix...
keywords            Verunglimpfung Nachtangriff Panzerspähwagen Re...
content             1. Luftkrieg über Le Havre, Frankreich, 1940 N...
Name: 0, dtype: object
   chapter                                            content
0        1  1. Luftkrieg über Le Havre, Frankreich, 1940 N...
1        2  2. Einmarsch in Amiens, Frankreich, 1940 Karte...
2        3  3. Einmar

0      None
1      None
2      None
3      None
4      None
       ... 
235    None
236    None
240    None
242    None
243    None
Length: 200, dtype: object